In [ ]:
#
# Stub Bundling and Confluent Spirals for Geographic Networks
# - Arlind Nocaj and Ulrik Brandes
# - Department of Computer & Information Science, University of Konstanz
# S. Wismath and A. Wolff (Eds.): GD 2013, LNCS 8242, pp. 388–399, 2013.
#
from math import pi, cos, sin, tan, atan2, exp, sqrt, copysign
import rtsvg
rt = rtsvg.RACETrack()

class SVGGridPaper(object):
    #
    # __init__() - constructor
    # ... creates grid paper and fills it in with the points objects
    # ... those objects should have a member variable called "pts" which is a list of (x,y) tuples
    #
    def __init__(self, labeled_points=None, w=512, h=512):
        self.w, self.h      = w, h
        self.labeled_points = labeled_points
        self.pts_objs       = []

    #
    # add() - add more points objects
    #
    def add(self, *pts):
        self.pts_objs.extend(pts)

    #
    # SVG Representation
    #
    def _repr_svg_(self):
        _pts_ = self.pts_objs[0]
        x0, y0, x1, y1 = _pts_.pts[0][0], _pts_.pts[0][1], _pts_.pts[0][0], _pts_.pts[0][1]
        for _pts_ in self.pts_objs:
            for _pt_ in _pts_.pts: x0, y0, x1, y1 = min(x0,_pt_[0]), min(y0,_pt_[1]), max(x1,_pt_[0]), max(y1,_pt_[1])            
        x_perc = 0.05 * (x1 - x0)
        y_perc = 0.05 * (y1 - y0)
        x0, y0, x1, y1 = x0 - x_perc, y0 - y_perc, x1 + x_perc, y1 + y_perc
        svg = [f'<svg x="0" y="0" width="{self.w}" height="{self.h}" viewBox="{x0} {y0} {x1-x0} {y1-y0}" xmlns="http://www.w3.org/2000/svg">']
        svg.append(f'<rect x="{x0}" y="{y0}" width="{x1-x0}" height="{y1-y0}" fill="#ffffff" />')
        x = int(x0)
        if int(x1) - int(x0) < 200:
            while x < x1:
                svg.append(f'<line x1="{x}" y1="{y0}" x2="{x}" y2="{y1}" stroke="#a0a0a0" stroke-width="0.05"/>')
                x += 1
        y = int(y0)
        if int(y1) - int(y0) < 200:
            while y < y1:
                svg.append(f'<line x1="{x0}" y1="{y}" x2="{x1}" y2="{y}" stroke="#a0a0a0" stroke-width="0.05"/>')
                y += 1
        svg.append(f'<line x1="{0.0}" y1="{y0}"  x2="{0.0}" y2="{y1}"  stroke="#a0a0a0" stroke-width="0.1"/>')
        svg.append(f'<line x1="{x0}"  y1="{0.0}" x2="{x1}"  y2="{0.0}" stroke="#a0a0a0" stroke-width="0.1"/>')

        if self.labeled_points is not None:
            for _label_ in self.labeled_points:
                _xy_ = self.labeled_points[_label_]
                svg.append(rt.svgText(_label_, _xy_[0], _xy_[1]-0.5, txt_h=1.5, color='#a0a0a0', anchor='middle'))
                svg.append(f'<circle cx="{_xy_[0]}" cy="{_xy_[1]}" r="0.2" fill="#ff0000"/>')

        for _pts_ in self.pts_objs:
            _path_ = [f'M {_pts_.pts[0][0]} {_pts_.pts[0][1]}']
            for i in range(1, len(_pts_.pts)): _path_.append(f'L {_pts_.pts[i][0]} {_pts_.pts[i][1]}')
            svg.append(f'<path d="{" ".join(_path_)}" fill="none" stroke="#000000" stroke-width="0.1"/>')
            for i in range(len(_pts_.pts)):
                _pt_ = _pts_.pts[i]
                svg.append(f'<circle cx="{_pt_[0]}" cy="{_pt_[1]}" r="0.1" fill="#000000"/>')
        svg.append('</svg>')
        return ''.join(svg)


In [ ]:
#
# ConfluentSpiral class
# - Created from Gemini 2.5 Pro
#
class ConfluentSpiral(object):
    #
    # Constructor
    #
    def __init__(self, pv, pw, theta=pi/3.0, t_inc=0.1, t_min=0.0, t_max=10.0):
        self.pv,    self.pw,    self.theta = pv,    pw,    theta
        self.t_inc, self.t_min, self.t_max = t_inc, t_min, t_max

        b = 1.0 / tan(-theta) if theta < 0.0 else 1.0 / tan( theta)
        u_wv_x = pv[0] - pw[0]
        u_wv_y = pv[1] - pw[1]
        L0     = sqrt(u_wv_x**2 + u_wv_y**2)
        phi0   = atan2(u_wv_y, u_wv_x)

        self.pts = []
        t        = t_min
        while t <= t_max:
            radial_factor = L0 * exp(-b * t)
            # Angle depends on the sign of b according to the paper's convention
            # Angle = phi0 + sgn(b) * t
            sign_b = copysign(1.0, theta) # Get sign of b (+1.0 or -1.0)
            current_angle = phi0 + sign_b * t
            # Calculate displacement vector components
            delta_x = radial_factor * cos(current_angle)
            delta_y = radial_factor * sin(current_angle)
            # Calculate the absolute coordinates by adding displacement to the pole
            x = pw[0] + delta_x
            y = pw[1] + delta_y
            self.pts.append((x,y))
            t += t_inc

_tiles_    = []
_pv_, _pw_ = (1.0, 2.0),(-17.0, 9.0)
for _pos_ in [1.0, -1.0]:
    svg_grid_paper = SVGGridPaper(labeled_points={'pv':_pv_, 'pw':_pw_})
    for _div_ in [2.5, 3.0, 5.0, 9.0, 21.0]: svg_grid_paper.add(ConfluentSpiral(_pv_, _pw_, theta=_pos_ * pi/_div_))
    _tiles_.append(svg_grid_paper)
rt.tile(_tiles_)